In [ ]:
%pip install -qU python-dotenv langchain-upstage langchain-pinecone langchain-text-splitters langchain_core python-docx langchain-openai langchain-ollama langchain-huggingface
%pip install --upgrade --quiet  docx2txt langchain_community
%pip install unstructured

In [1]:
from dotenv import load_dotenv 
from langchain_upstage import UpstageEmbeddings

embedding = UpstageEmbeddings(model="solar-embedding-1-large")
load_dotenv()

True

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, UnstructuredWordDocumentLoader

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200,
)

loader = DirectoryLoader("temp", glob="*.docx", loader_cls=UnstructuredWordDocumentLoader)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list

In [ ]:
%pip install -U langchain-ollama

In [25]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "Llama-3-Open-Ko-8B-Q8_0",
    temperature = 0.8,
    num_predict = 256,
)
llm.invoke("hi")

AIMessage(content='hello', response_metadata={'model': 'Llama-3-Open-Ko-8B-Q8_0', 'created_at': '2024-09-04T06:01:30.9923398Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 11496523200, 'load_duration': 6351412600, 'prompt_eval_count': 54, 'prompt_eval_duration': 4348779000, 'eval_count': 4, 'eval_duration': 791459000}, id='run-3f7794bf-3426-4451-be6f-fe4b27b584e3-0', usage_metadata={'input_tokens': 54, 'output_tokens': 4, 'total_tokens': 58})

In [ ]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'vtw-2'

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [30]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="Llama-3-Open-Ko-8B-Q8_0", base_url=None)
database = PineconeVectorStore.from_existing_index(index_name="vtw-2", embedding=embedding)
database.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OllamaEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000210362CC9E0>)